In [3]:
# not all of these libraries are needed but for now let's load them to be prepared :-) 
from ipyleaflet import (Map, GeoData, WidgetControl, GeoJSON, basemaps,
 LayersControl, Icon, Marker,basemap_to_tiles, Choropleth, AntPath,
 MarkerCluster, Heatmap, SearchControl, FullScreenControl, AwesomeIcon, 
 ScaleControl, MeasureControl, SplitMapControl, WMSLayer, Polygon, Choropleth)

In [4]:
# ipywidgets add user interactions to our notebook cells 
# read the docs https://ipywidgets.readthedocs.io/en/latest/
from ipywidgets import Text, HTML, IntSlider, ColorPicker, jslink, Layout
from branca.colormap import linear

In [5]:
# something to look into when wanting to color areas of a city or a region (Choropleth Maps)
# https://blog.datawrapper.de/choroplethmaps/
#import geopandas as gpd
#import json

In [6]:
# Pandas = derived from "Python and data analysis"
import pandas as pd  

## Basic Map Parameters 

In [9]:
# the centre of your map should be about the starting point of your mapping tour 
center = [48.231139, 16.374955]
zoom =16

# you can adjust the map size via ipwidgets *Layout* attribute  
basemap = basemaps.Esri.WorldStreetMap
layout = Layout(width='100%', height='600px')

"""
alternative options for basemaps are 
basemap = basemaps.Stamen.Watercolor
basemap = basemaps.Stamen.Toner
basemap = basemaps.Stamen.Terrain
etc 

"""

'\nalternative options for basemaps are \nbasemap = basemaps.Stamen.Watercolor\nbasemap = basemaps.Stamen.Toner\nbasemap = basemaps.Stamen.Terrain\netc \n\n'

## Basic Map 

In [12]:
m = Map(center=center, zoom=zoom, basemap = basemap, layout=layout)

# add user interaction / user information such as the scale of a map 
zoom_slider = IntSlider(description='Zoom level:', min=10, max=20, value=16)
jslink((zoom_slider, 'value'), (m, 'zoom'))

widget_control1 = WidgetControl(widget=zoom_slider, position='topright')
m.add_control(widget_control1)
m.add_control(FullScreenControl())
m.add_control(ScaleControl(position='bottomleft', imperial = False))
m.add_control(LayersControl(position='topright'))

# this adds a nice feature to meassure the length of a path or a polygone area in square meters 
measure = MeasureControl(
    position='bottomleft',
    active_color = 'orange',
    primary_length_unit = 'meters',
    primary_area_unit = 'sqmeters',
    completed_color = 'blue'
)
m.add_control(measure)
display (m)

Map(center=[48.231139, 16.374955], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

## Create a Split Map 

In [17]:
basemap = basemaps.Esri.WorldStreetMap
split_map = Map(center=center, zoom=zoom, layout=layout)

# create right and left layers
left_layer  = basemap_to_tiles(basemap=basemap)
right_layer = basemap_to_tiles(basemap=basemaps.Stamen.Toner) 

# create split control
control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)

#add control to map
split_map.add_control(control)

# display map

zoom_slider = IntSlider(description='Zoom level:', min=10, max=20, value=16)
jslink((zoom_slider, 'value'), (m, 'zoom'))

widget_control1 = WidgetControl(widget=zoom_slider, position='topright')
split_map.add_control(widget_control1)
split_map.add_control(FullScreenControl())
split_map.add_control(ScaleControl(position='bottomleft', imperial = False))
split_map.add_control(LayersControl(position='topright')) 

display(split_map)

Map(center=[48.231139, 16.374955], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

## Reading the data as provided by the gadget (there are always two types of files - tracks and ratings of spots)

In [19]:
ratings=pd.read_csv('data/2020-10-12_rate_waste.csv', sep=';')
tracks=pd.read_csv('data/2020-10-12_track_waste.csv', sep=';')
print('number of track points: ', len (tracks))
tracks.head(3)

number of track points:  500


,Date,Time,Raw_Time,Latitude,Longitude,Altitude,Sats,Sat_Speed,Precision
0,121020,07:12:47,6124700,48.2311,16.375006,160.2,4,0.5926,6.51
1,121020,07:12:47,6124700,48.2311,16.375006,160.2,4,0.5926,6.51
2,121020,07:12:47,6124700,48.2311,16.375006,160.2,4,0.5926,6.51


## Frequency of categories 1 - 5 

In [20]:
ratings.groupby(['Category']).size()

Category
1    13
2     4
4     5
dtype: int64

In [21]:
# at this point we add some meaning to the categories 
tree_waste = ratings[ (ratings['Category'] == 1) | (ratings['Category'] == 2) ] 
bin_waste = ratings[ ratings['Category'] == 4] 
print ('number of polutted trees: ', len(tree_waste))
print ('number of overflowing bins: ', len(bin_waste))

number of polutted trees:  17
number of overflowing bins:  5


In [22]:
# a collection of points (GPS coordinates) needs to be provided as a list of lists, this is what this funciton does  
def location_converter (df): 
    markers = df.loc[:,{'Latitude','Longitude'}] #ouput dataframe 
    markers = markers.reindex(columns = ['Latitude','Longitude']) #
    markers = markers.to_records(index=False) #output array
    markers = list (markers) #output list of tuples
    markers = [list(i) for i in markers] # list of lists
    return markers


In [24]:
# calling the funnnction and checking outpu 
tree_pos = location_converter (tree_waste)
bin_pos = location_converter (bin_waste)
track_pos = location_converter (tracks)
track_pos [0:2]

[[48.2311, 16.375006], [48.2311, 16.375006]]

In [25]:
import os
os.getcwd()

'/Users/me/code/notebooks/ipyleaflet'

## Adding tree- and rubish bin locations to the map 

In [26]:
#trash_icon = AwesomeIcon (name='trash', marker_color='white', icon_color='black', spin=True)
#trash_icon = Icon(icon_url='https://leafletjs.com/examples/custom-icons/leaf-red.png', icon_size=[25, 40]) #red, green, yellow

bin_markers = []
tree_markers = []
foot_markers = []

trash_icon = Icon(icon_url='files/'+os.getcwd().split('/')[-1]+'/icons/trashbin.png', icon_size=[30, 40])
tree_icon = Icon(icon_url='files/'+os.getcwd().split('/')[-1]+'/icons/tree.png', icon_size=[30, 40])
foot_icon = Icon(icon_url='files/'+os.getcwd().split('/')[-1]+'/icons/foot.png', icon_size=[30, 40])


for i in range(len(bin_pos)):
    marker  = Marker(location=bin_pos[i], icon = trash_icon)
    m.add_layer(marker);
    #bin_markers = bin_markers + marker
    
    
for i in range(len(tree_pos)):
    marker  = Marker(location=tree_pos[i], icon = tree_icon)
    m.add_layer(marker);
    #tree_markers = tree_markers + marker
    
'''
for i in range(len(track_pos)):
    marker  = [Marker(location=track_pos[i], icon = foot_icon)]
    foot_markers = foot_markers + marker
   

bin_markers = tuple (bin_markers)
tree_markers = tuple (tree_markers)
#foot_markers = tuple (foot_markers)
''' 


Icon(icon_size=[30, 40], icon_url='files/ipyleaflet/icons/tree.png', options=['icon_anchor', 'icon_size', 'icon_url', 'popup_anchor', 'shadow_anchor', 'shadow_size', 'shadow_url'], popup_anchor=[0, 0])


'\nfor i in range(len(track_pos)):\n    marker  = [Marker(location=track_pos[i], icon = foot_icon)]\n    foot_markers = foot_markers + marker\n   \n\nbin_markers = tuple (bin_markers)\ntree_markers = tuple (tree_markers)\n#foot_markers = tuple (foot_markers)\n'

In [29]:
# creating the path of the mapping exercise 

ant_path = AntPath (
    locations=track_pos,     
    dash_array=[1, 10],
    delay=2000,
    color='#7590ba',
    pulse_color='#3f6fba',
    name='Trail')

m.add_layer(ant_path)
m.layout = Layout(width='100%', height='400px')

display (m) 

Map(bottom=5816062.0, center=[48.23264883085179, 16.3747251033783], controls=(ZoomControl(options=['position',…

## Grouping of markers (not really useful in this context) 

In [31]:
"""
bin_layer = MarkerCluster(markers = bin_markers, name='Trash bins')
tree_layer = MarkerCluster(markers = tree_markers, name='Tree w. trash')

#foot_layer = MarkerCluster(markers = foot_markers, name='Trail')
# print(bin_layer)

m.add_layer(bin_layer) 
m.add_layer(tree_layer) 
m.add_layer(ant_path)
m
"""


"\nbin_layer = MarkerCluster(markers = bin_markers, name='Trash bins')\ntree_layer = MarkerCluster(markers = tree_markers, name='Tree w. trash')\n\n#foot_layer = MarkerCluster(markers = foot_markers, name='Trail')\n# print(bin_layer)\n\nm.add_layer(bin_layer) \nm.add_layer(tree_layer) \nm.add_layer(ant_path)\nm\n"

## The resulting HTML file should be visible in any browser (however, icons will be missing - fixible)

In [32]:
m.save('my_map.html', title='My Map')

## Some cleaning up if needed 

In [ ]:
m.clear_layers()

In [ ]:
m.remove_layer(bin_layer)
m.remove_layer(tree_layer)
m.remove_layer (ant_path)

## Integrating a different basemap with more details 

In [34]:
from ipyleaflet import Map, WMSLayer, basemaps
#wmts = "http://maps.wien.gv.at/basemap/geolandbasemap/normal/google3857/{z}/{y}/{x}.png"

wms = WMSLayer(
    url='http://maps.wien.gv.at/basemap/geolandbasemap/normal/google3857/{z}/{y}/{x}.png',
    format='image/png',
    transparent=True,
    attribution='wait'
)

m.add_layer(wms)

m

Map(bottom=5815943.0, center=[48.232920399369064, 16.37470364570618], controls=(ZoomControl(options=['position…

## For later: Experimenting with coloring regions or neighborhoods 

In [35]:
import csv 
from collections import defaultdict

In [36]:
#the syntax is: mydict[key] = "value"
#mydict ["iphone 5S"] = 2013

def parse_csv_by_field(filename, fieldnames):
    print(fieldnames)
    d = defaultdict(list)
    with open(filename, newline='') as csvfile:
        reader = csv.DictReader(csvfile, fieldnames)
        next(reader)  # remove header
        for row in reader:
            d[row ['bundesland']] = int (row ['measurement'])
    return dict(d)


area_data = parse_csv_by_field('data/area_data.csv', ['bundesland','measurement'])
area_data 
 
    

['bundesland', 'measurement']


{}

In [37]:
m.clear_layers()

In [42]:
import geopandas as gpd
import json 
states = gpd.read_file('geojson/laender.json')
print(states.head())

  iso              name                                           geometry
0   4    Oberösterreich  POLYGON ((13.76147 47.52081, 13.74994 47.50213...
1   7             Tirol  MULTIPOLYGON (((12.71233 46.73610, 12.71151 46...
2   6        Steiermark  POLYGON ((14.07995 47.02247, 14.01723 46.98951...
3   1        Burgenland  POLYGON ((16.06678 46.85025, 15.99729 46.83575...
4   3  Niederösterreich  POLYGON ((14.85902 47.74099, 14.81927 47.74470...


In [44]:
geo_json_borders ['features'] [0] ['properties'] ['name']

'Oberösterreich'